In [2]:
!pip install mysql-connector-python
!python connect_to_mysql.py


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 35.5 MB/s eta 0:00:00
python3: can't open file '/content/connect_to_mysql.py': [Errno 2] No such file or directory


In [3]:
import mysql.connector
from mysql.connector import Error
from datetime import datetime

# Database connection details
host = "fyp24-wasaysattar033-a818.b.aivencloud.com"  # Replace with your database host
port = 21550  # Replace with your database port (integer)
user = ""  # Replace with your username
password = ""  # Replace with your password
database = "defaultdb"  # Replace with your database name

def print_table(data, column_names):
    # Printing the table with headers and data
    column_widths = [max(len(str(value)) for value in column) for column in zip(*data, column_names)]
    row_format = " | ".join([f"{{:<{width}}}" for width in column_widths])

    print(f"+{'-' * (sum(column_widths) + 3 * (len(column_names) - 1))}+")
    print(row_format.format(*column_names))
    print(f"+{'-' * (sum(column_widths) + 3 * (len(column_names) - 1))}+")

    for row in data:
        print(row_format.format(*row))

    print(f"+{'-' * (sum(column_widths) + 3 * (len(column_names) - 1))}+")

def check_date_differences(cursor, table_name):
    """Check if dates in the specified table are within 7 days of the current date."""
    cursor.execute(f"SELECT IOC, Date FROM {table_name};")
    rows = cursor.fetchall()
    current_date = datetime.now().date()

    for row in rows:
        ioc, date = row
        if date:
            date_difference = (current_date - date).days
            if abs(date_difference) <= 7:
                print(f"{ioc}: Date is within 7 days.")
            else:
                print(f"{ioc}: Date has exceeded 7 days.")
        else:
            print(f"{ioc}: No date available.")

try:
    # Establishing the connection
    connection = mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database
    )

    if connection.is_connected():
        print("Connected to the database successfully!")
        cursor = connection.cursor()

        # SQL statements for creating tables
        create_ioc_data = """
        CREATE TABLE IF NOT EXISTS IOC_Data (
            IOC VARCHAR(15) PRIMARY KEY,
            Severity VARCHAR(10),
            Threat_Type VARCHAR(20),
            Reputation VARCHAR(20),
            Date DATE
        );
        """
        create_severity_ti = """
        CREATE TABLE IF NOT EXISTS Severity_TI (
            IOC VARCHAR(15) PRIMARY KEY,
            AbuseIP VARCHAR(10),
            VirusTotal VARCHAR(10),
            IBM_Xforce VARCHAR(10),
            AnyRun VARCHAR(10)
        );
        """
        create_threat_type_ti = """
        CREATE TABLE IF NOT EXISTS Threat_Type_TI (
            IOC VARCHAR(15) PRIMARY KEY,
            AbuseIP VARCHAR(20),
            VirusTotal VARCHAR(20),
            IBM_Xforce VARCHAR(20),
            AnyRun VARCHAR(20)
        );
        """
        create_reputation_ti = """
        CREATE TABLE IF NOT EXISTS Reputation_TI (
            IOC VARCHAR(15) PRIMARY KEY,
            AbuseIP VARCHAR(15),
            VirusTotal VARCHAR(15),
            IBM_Xforce VARCHAR(15),
            AnyRun VARCHAR(20)
        );
        """
        create_admin_credentials = """
        CREATE TABLE IF NOT EXISTS Admin_Credentials (
            ID INT PRIMARY KEY,
            First_Name VARCHAR(20),
            Last_Name VARCHAR(20),
            Contact VARCHAR(15),
            Email VARCHAR(50),
            Password VARCHAR(20)
        );
        """
        create_user_credentials = """
        CREATE TABLE IF NOT EXISTS User_Credentials (
            ID INT PRIMARY KEY,
            First_Name VARCHAR(20),
            Last_Name VARCHAR(20),
            Contact VARCHAR(15),
            Email VARCHAR(50),
            Password VARCHAR(20)
        );
        """

        # Executing table creation queries
        cursor.execute(create_ioc_data)
        print("IOC_Data table created successfully.")
        cursor.execute(create_severity_ti)
        print("Severity_TI table created successfully.")
        cursor.execute(create_threat_type_ti)
        print("Threat_Type_TI table created successfully.")
        cursor.execute(create_reputation_ti)
        print("Reputation_TI table created successfully.")
        cursor.execute(create_admin_credentials)
        print("Admin_Credentials table created successfully.")
        cursor.execute(create_user_credentials)
        print("User_Credentials table created successfully.")

        # SQL statements for inserting data with ON DUPLICATE KEY UPDATE
        insert_ioc_data = """
        INSERT INTO IOC_Data (IOC, Severity, Threat_Type, Reputation, Date) VALUES
        ('10.0.0.5', 'Low', 'Test Malware', 'Benign', '2024-10-21'),
        ('172.16.0.2', 'Medium', 'Phishing', 'Known Malicious', '2024-11-02'),
        ('192.168.10.5', 'High', 'Malware', 'Suspicious', '2024-10-29'),
        ('203.0.113.8', 'Critical', 'Ransomware', 'Malicious', '2024-10-19')
        ON DUPLICATE KEY UPDATE
        Severity=VALUES(Severity), Threat_Type=VALUES(Threat_Type), Reputation=VALUES(Reputation), Date=VALUES(Date);
        """
        insert_severity_ti = """
        INSERT INTO Severity_TI (IOC, AbuseIP, VirusTotal, IBM_Xforce, AnyRun) VALUES
        ('10.0.0.5', 'Low', 'Low', 'Medium', 'High'),
        ('172.16.0.2', 'Medium', 'Low', 'High', 'Critical'),
        ('192.168.10.5', 'High', 'Medium', 'Low', 'Critical'),
        ('203.0.113.8', 'Critical', 'High', 'Medium', 'Low')
        ON DUPLICATE KEY UPDATE
        AbuseIP=VALUES(AbuseIP), VirusTotal=VALUES(VirusTotal), IBM_Xforce=VALUES(IBM_Xforce), AnyRun=VALUES(AnyRun);
        """
        insert_threat_type_ti = """
        INSERT INTO Threat_Type_TI (IOC, AbuseIP, VirusTotal, IBM_Xforce, AnyRun) VALUES
        ('10.0.0.5', 'Adware', 'Test Malware', 'Spyware', 'Keylogger'),
        ('172.16.0.2', 'Botnet', 'Trojan', 'Worm', 'Ransomware'),
        ('192.168.10.5', 'Malware', 'Phishing', 'Spam', 'Spyware'),
        ('203.0.113.8', 'Ransomware', 'Malware', 'Backdoor', 'Phishing')
        ON DUPLICATE KEY UPDATE
        AbuseIP=VALUES(AbuseIP), VirusTotal=VALUES(VirusTotal), IBM_Xforce=VALUES(IBM_Xforce), AnyRun=VALUES(AnyRun);
        """
        insert_reputation_ti = """
        INSERT INTO Reputation_TI (IOC, AbuseIP, VirusTotal, IBM_Xforce, AnyRun) VALUES
        ('10.0.0.5', 'Suspicious', 'Benign', 'Malicious', 'Unknown'),
        ('172.16.0.2', 'Benign', 'Malicious', 'Unknown', 'Benign'),
        ('192.168.10.5', 'Malicious', 'Benign', 'Suspicious', 'Known Malicious'),
        ('203.0.113.8', 'Unknown', 'Malicious', 'Suspicious', 'Benign')
        ON DUPLICATE KEY UPDATE
        AbuseIP=VALUES(AbuseIP), VirusTotal=VALUES(VirusTotal), IBM_Xforce=VALUES(IBM_Xforce), AnyRun=VALUES(AnyRun);
        """
        insert_admin_credentials = """
        INSERT INTO Admin_Credentials (ID, First_Name, Last_Name, Contact, Email, Password) VALUES
        (1, 'Abdul', 'Wasay', '0987654321', 'abdul@example.com', 'letmein101'),
        (2, 'Muhammad', 'Shaheer', '1234567890', 'muhammad@example.com', 'pass123'),
        (3, 'Taha', 'Khan', '5566778899', 'taha@example.com', 'qwerty456'),
        (4, 'Qirat', 'Saleem', '1122334455', 'qirat@example.com', 'admin789')
        ON DUPLICATE KEY UPDATE
        First_Name=VALUES(First_Name), Last_Name=VALUES(Last_Name), Contact=VALUES(Contact), Email=VALUES(Email), Password=VALUES(Password);
        """
        insert_user_credentials = """
        INSERT INTO User_Credentials (ID, First_Name, Last_Name, Contact, Email, Password) VALUES
        (1, 'Abdul', 'Wasay', '0987654321', 'abdul@example.com', 'letmein101'),
        (2, 'Muhammad', 'Shaheer', '1234567890', 'muhammad@example.com', 'pass123'),
        (3, 'Taha', 'Khan', '5566778899', 'taha@example.com', 'qwerty456'),
        (4, 'Qirat', 'Saleem', '1122334455', 'qirat@example.com', 'admin789')
        ON DUPLICATE KEY UPDATE
        First_Name=VALUES(First_Name), Last_Name=VALUES(Last_Name), Contact=VALUES(Contact), Email=VALUES(Email), Password=VALUES(Password);
        """

        # Executing insert queries
        cursor.execute(insert_ioc_data)
        cursor.execute(insert_severity_ti)
        cursor.execute(insert_threat_type_ti)
        cursor.execute(insert_reputation_ti)
        cursor.execute(insert_admin_credentials)
        cursor.execute(insert_user_credentials)

        # Checking date differences
        check_date_differences(cursor, 'IOC_Data')

        # Retrieving and displaying table data
        tables = [
            ('IOC_Data', ['IOC', 'Severity', 'Threat_Type', 'Reputation', 'Date']),
            ('Severity_TI', ['IOC', 'AbuseIP', 'VirusTotal', 'IBM_Xforce', 'AnyRun']),
            ('Threat_Type_TI', ['IOC', 'AbuseIP', 'VirusTotal', 'IBM_Xforce', 'AnyRun']),
            ('Reputation_TI', ['IOC', 'AbuseIP', 'VirusTotal', 'IBM_Xforce', 'AnyRun']),
            ('Admin_Credentials', ['ID', 'First_Name', 'Last_Name', 'Contact', 'Email', 'Password']),
            ('User_Credentials', ['ID', 'First_Name', 'Last_Name', 'Contact', 'Email', 'Password'])
        ]

        for table_name, columns in tables:
            cursor.execute(f"SELECT * FROM {table_name}")
            rows = cursor.fetchall()
            print(f"\n{table_name}:")
            print_table(rows, columns)

except Error as e:
    print(f"Error while connecting to MySQL: {e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed.")


Connected to the database successfully!
IOC_Data table created successfully.
Severity_TI table created successfully.
Threat_Type_TI table created successfully.
Reputation_TI table created successfully.
Admin_Credentials table created successfully.
User_Credentials table created successfully.
10.0.0.5: Date has exceeded 7 days.
172.16.0.2: Date has exceeded 7 days.
192.168.10.5: Date has exceeded 7 days.
203.0.113.8: Date has exceeded 7 days.

IOC_Data:
+---------------------------------------------------------------------+
IOC          | Severity | Threat_Type  | Reputation      | Date      
+---------------------------------------------------------------------+
10.0.0.5     | Low      | Test Malware | Benign          | <10
172.16.0.2   | Medium   | Phishing     | Known Malicious | <10
192.168.10.5 | High     | Malware      | Suspicious      | <10
203.0.113.8  | Critical | Ransomware   | Malicious       | <10
+---------------------------------------------------------------------+

Seve